### Import necessary libraries, packages, and dependencies 
- Use os to access the operating system
- Use pandas to create DataFrames and perform analysis 
- Use plotly to generate visuals
- Use dotenv to access .env variables 
- Use sqlalchemy to connect to postgres

In [1]:
import os
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Connect to database and retrieve necessary data
- Connect to the 'arbi_db' database 
- Retrieve all data from the 'control' table

In [30]:
# Load environment variables from .env file
load_dotenv()

# Access the variables 
psql_username = os.getenv('PSQL_USERNAME')
psql_password = os.getenv('PSQL_PASSWORD')
psql_host = os.getenv('PSQL_HOST')
psql_port = os.getenv('PSQL_PORT')
db_name = os.getenv('DB_NAME')

# Define the database url
db_url = f"postgresql://{psql_username}:{psql_password}@{psql_host}:{psql_port}/{db_name}" 

# Create the engine object
engine = create_engine(db_url)

# Write ethe SQL query
query = 'SELECT * FROM control'

# Read the SQL query into a DataFrame
control_df = pd.read_sql(query, engine)

# Show the DataFrame's head
control_df.head()


,trade_count,current_datetime,currency,volume,buy_exchange,buy_price,total_purchase_amount,sell_exchange,sell_price,total_sale_amount,profit,spread_percentage
0,1,2024-02-27T10:45:46.898027,bitcoin,1.0,Binance,56908.33,56908.33,Gemini,56930.57,56930.57,22.24,0.039080
1,2,2024-02-27T10:45:46.898027,bitcoin,1.0,Bitstamp,56921.00,56921.00,Gemini,56930.57,56930.57,9.57,0.016813
2,3,2024-02-27T10:45:46.898027,ethereum,1.0,Binance,3245.35,3245.35,Poloniex,3245.40,3245.40,0.05,0.001541
3,4,2024-02-27T10:45:46.898027,ethereum,1.0,Bitstamp,3244.80,3244.80,Poloniex,3245.40,3245.40,0.60,0.018491
4,5,2024-02-27T10:45:46.898027,ethereum,1.0,Gemini,3244.86,3244.86,Poloniex,3245.40,3245.40,0.54,0.016642


### Data Analysis
- Profit Distribution across Trades
- Hourly Profit Trend
- Spread Percentage Analysis 
- Exchange Performance 
- Time Series Analysis

-Note: Limited data collection - Program ran every 30 seconds over a 4 hour window

In [31]:
# Convert the current_datetime column to only show the date
control_df['current_datetime'] = pd.to_datetime(control_df['current_datetime'])
control_df['current_datetime'] = control_df['current_datetime'].apply(lambda x: x.time())

# Set the index to the current_datetime column
control_df.set_index('current_datetime', inplace=True)

control_df

,trade_count,currency,volume,buy_exchange,buy_price,total_purchase_amount,sell_exchange,sell_price,total_sale_amount,profit,spread_percentage
current_datetime,,,,,,,,,,,
10:45:46.898027,1,bitcoin,1.0,Binance,56908.3300,56908.3300,Gemini,56930.57000,56930.57000,22.24000,0.039080
10:45:46.898027,2,bitcoin,1.0,Bitstamp,56921.0000,56921.0000,Gemini,56930.57000,56930.57000,9.57000,0.016813
10:45:46.898027,3,ethereum,1.0,Binance,3245.3500,3245.3500,Poloniex,3245.40000,3245.40000,0.05000,0.001541
10:45:46.898027,4,ethereum,1.0,Bitstamp,3244.8000,3244.8000,Poloniex,3245.40000,3245.40000,0.60000,0.018491
10:45:46.898027,5,ethereum,1.0,Gemini,3244.8600,3244.8600,Poloniex,3245.40000,3245.40000,0.54000,0.016642
...,...,...,...,...,...,...,...,...,...,...,...
14:45:44.441654,3983,polygon,1.0,Kraken,1.0266,1.0266,Poloniex,1.02683,1.02683,0.00023,0.022404
14:45:44.441654,3984,solana,1.0,Binance,108.0400,108.0400,Bitstamp,108.06430,108.06430,0.02430,0.022492
14:45:44.441654,3985,solana,1.0,Binance,108.0400,108.0400,Gemini,108.16300,108.16300,0.12300,0.113847


##### Profit Distribution across Trades
- Calculate and visualize the distribution of profits across all trades. This provides insights into the consistency and variability of returns. 

In [35]:
profit_dist = px.histogram(
    control_df, 
    x='profit', 
    nbins=20, 
    title='Profit Distribution', 
    labels={'profit': 'Profit', 'count':'Frequency'})

profit_dist.show()

##### 30 Sec Profit Trend
- Examine the 30 second trend of profits to identify patterns and understand the overall performance of the arbitrage over time.

In [38]:
thirty_sec_profit = control_df.groupby('current_datetime')['profit'].sum().reset_index()
thirty_sec_profit_plot = px.line(
    thirty_sec_profit, 
    x='current_datetime',
    y='profit', 
    title='30 Second Profit Trend',
    labels={'current_datetime':'Time', 'profit':'30 Sec Profit'})

thirty_sec_profit_plot.show()

##### Spread Percentage Analysis
- Explore the spread percentage to understand how much profit is generated relative to the spread. Helps to access the efficieny of the arbitrage strategy. 

In [39]:
spread_percentage_plot = px.scatter(
    control_df, 
    x='spread_percentage',
    y='profit',
    title='Spread Percentage vs Profit',
    labels={'spread_percentage':'Spread Percentage', 'profit':'Profit'})

spread_percentage_plot.show()

##### Exchange Performance
- Evaluate the performance of each exchange in terms of the number of trades and profit. This helps in optimizing the selection of exchanges and diversifying risks.

In [40]:
exchange_performance = control_df.groupby('buy_exchange')['profit'].agg(['count', 'sum']).reset_index()
exchange_performance_plot = px.bar(
    exchange_performance,
    x='buy_exchange',
    y=['count', 'sum'],
    barmode='stack',
    title='Exchange Performance',
    labels={'buy_exchange':'Exchange', 'value':'Value'}
)

exchange_performance_plot.show()

##### Time Series Analysis
- Analyze the time series data to identify any patterns or trends in the arbitrage opportunities.

In [45]:
time_series = control_df.reset_index()

time_series_plot = px.line(
    time_series,
    x='current_datetime', 
    y='profit', 
    title='Time Series Analysis',
    labels={'current_datetime': 'Time', 'profit':'Profit'}
)
time_series_plot.show()

### Calculate a minimum profit threshold
- Add transaction fees (use 0.1 percent)
- Adjust the spread_percentage to account for fees

In [47]:
# Calculate total transaction cost for each trade
control_df['buy_fee'] = 0.1  
control_df['sell_fee'] = 0.1  
control_df['total_transaction_cost'] = control_df['buy_fee'] + control_df['sell_fee']

# Filter trades based on a minimum profit threshold in percentage
# Create a list of thresholds values at (1%, 2%, 3%, 4%, 5%)
thresholds = [1, 2, 3, 4, 5]

# Create an empty dictionary to store total profits for each threshold
threshold_profits = {}

# Loop through each threshold and calculate profits
for threshold in thresholds:
    # Adjust spread percentage for expected fees
    control_df['adjusted_spread_percentage'] = control_df['spread_percentage'] - control_df['total_transaction_cost']
    
    # Filter trades by threshold
    profitable_trades = control_df[control_df['adjusted_spread_percentage'] > threshold]

    # Calculate profits 
    profitable_trades['profit'] = profitable_trades['sell_price'] - profitable_trades['buy_price'] - profitable_trades['total_transaction_cost']
    total_profits = profitable_trades['profit'].sum()

    # Store results to dictionary 
    threshold_profits[f'{threshold}% Threshold'] = total_profits

/var/folders/wl/x3hxw0v13sb9sgxcqzzdxwlh0000gn/T/ipykernel_15378/3669154984.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
# View profits at various thresholds
threshold_profits

{'1% Threshold': 19.23770000000002,
 '2% Threshold': 0.0,
 '3% Threshold': 0.0,
 '4% Threshold': 0.0,
 '5% Threshold': 0.0}